In [1]:
%%bash 

today=$(date '+%Y-%m-%d')

# wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/alternative \
#     -O gwas_associations_${today}.tsv
    
# wget -q https://www.ebi.ac.uk/gwas/api/search/downloads/studies_alternative \
#     -O gwas_studies_${today}.tsv
    
wget -q  https://www.ebi.ac.uk/gwas/api/search/downloads/ancestry    \
    -O gwas_ancestries_${today}.tsv

ls -lah *tsv

-rw-r--r--  1 dsuveges  384566875    10M 23 May 22:06 gwas_ancestries_2022-05-23.tsv
-rw-r--r--  1 dsuveges  384566875   227M 20 May 06:31 gwas_associations_2022-05-20.tsv
-rw-r--r--  1 dsuveges  384566875   227M 23 May 16:41 gwas_associations_2022-05-23.tsv
-rw-r--r--  1 dsuveges  384566875    17M 20 May 06:31 gwas_studies_2022-05-20.tsv
-rw-r--r--  1 dsuveges  384566875    17M 23 May 16:41 gwas_studies_2022-05-23.tsv


In [1]:
import pyspark.sql
import pyspark.sql.types as t
import pyspark.sql.functions as f

global spark

spark = (pyspark.sql.SparkSession
       .builder
       .master("local[*]")
       .getOrCreate()
)
        
# Reading GWAS Catalog data into spark dataframe
gwas_df = spark.read.csv('gwas_associations_2022-05-20.tsv', sep='\t', header=True).persist()

print(gwas_df.count())
gwas_df.show(1,False,True)
gwas_df.printSchema()


372752
-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 DATE ADDED TO CATALOG      | 2019-03-18                                                                                                                                                    
 PUBMEDID                   | 30643258                                                                                                                                                      
 FIRST AUTHOR               | Karlsson Linner R                                                                                                                                             
 DATE                       | 2019-01-14                                                                                                                                                    
 JOURNAL                    | Nat Genet         

In [32]:
print('Study count: {}'.format(gwas_df.select('STUDY ACCESSION').distinct().count()))
print('Split count by variants:')
print('   By CHR_ID: {}'.format(gwas_df.filter(f.col('CHR_ID').contains(';')).count()))
print('   By CHR_POS: {}'.format(gwas_df.filter(f.col('CHR_POS').contains(';')).count()))
print('   By CHR_ID or CHR_POS: {}'.format(gwas_df.filter(f.col('CHR_ID').contains(';') | f.col('CHR_POS').contains(';')).count()))
print('   By SNPS: {}'.format(gwas_df.filter(f.col('SNPS').contains(';')).count()))
print('   By STRONGEST SNP-RISK ALLELE: {}'.format(gwas_df.filter(f.col('STRONGEST SNP-RISK ALLELE').contains(';')).count()))



Study count: 15844
Split count by variants:
   By CHR_ID: 220
   By CHR_POS: 220
   By CHR_ID or CHR_POS: 220
   By SNPS: 1956
   By STRONGEST SNP-RISK ALLELE: 1956


In [47]:
(
    gwas_df
    .filter(f.col('CHR_POS').contains(';'))
    .select(
        f.split(f.col('CHR_ID'), ';').alias('CHR'), 
        f.split(f.col('CHR_POS'), ';').alias('POS'), 
        f.split(f.col("SNPS"), ';').alias('SNPS'),
        f.col('SNP_ID_CURRENT'),
        f.col('STUDY ACCESSION'),
        f.col('STRONGEST SNP-RISK ALLELE')
    )
    .filter(f.size('POS') != f.size('SNPS'))
    .show(1, vertical=True, truncate=False)
)

-RECORD 0-----------------------------------------------------------------------
 CHR                       | [6, 6]                                             
 POS                       | [32353777, 32421871]                               
 SNPS                      | [rs2395148,  rs2856683,  rs3135363,  rs9357152]    
 SNP_ID_CURRENT            | null                                               
 STUDY ACCESSION           | GCST000408                                         
 STRONGEST SNP-RISK ALLELE | rs2395148-A; rs2856683-C; rs3135363-A; rs9357152-A 
only showing top 1 row



In [48]:
(
    gwas_df
    .filter(
        f.col('SNPS').contains(';')
        & f.col('CHR_POS').isNull()
    )
    .select('CHR_ID', 'CHR_POS', 'SNPS', 'STUDY ACCESSION', 'SNP_ID_CURRENT', f.col('STRONGEST SNP-RISK ALLELE'))
    .show(3, False, True)
)

-RECORD 0-------------------------------------------------------------------
 CHR_ID                    | null                                           
 CHR_POS                   | null                                           
 SNPS                      | rs17435444; rs13116936                         
 STUDY ACCESSION           | GCST001879                                     
 SNP_ID_CURRENT            | null                                           
 STRONGEST SNP-RISK ALLELE | rs17435444-A; rs13116936-G                     
-RECORD 1-------------------------------------------------------------------
 CHR_ID                    | null                                           
 CHR_POS                   | null                                           
 SNPS                      | SNP_A-2032347; SNP_A-2196879; rs10871290       
 STUDY ACCESSION           | GCST000187                                     
 SNP_ID_CURRENT            | null                                           

In [16]:
(
    gwas_df
    .filter(
        f.col('CHR_POS').contains(';')
    )
    .show(3, False, True)
)

-RECORD 0-----------------------------------------------------------------------------------------------------------------------------------------------------
 DATE ADDED TO CATALOG      | 2011-01-14                                                                                                                      
 PUBMEDID                   | 20801717                                                                                                                        
 FIRST AUTHOR               | Shatunov A                                                                                                                      
 DATE                       | 2010-10-01                                                                                                                      
 JOURNAL                    | Lancet Neurol                                                                                                                   
 LINK                       | www.ncbi.nlm.nih

In [22]:
#     # Split rows by multiple variants
#     gwas['CHR_ID'] = gwas['CHR_ID'].astype(str).str.split(';')
#     gwas['CHR_POS'] = gwas['CHR_POS'].astype(str).str.split(';')
#     gwas['SNPS'] = gwas['SNPS'].astype(str).str.split('; ')
#     gwas['STRONGEST SNP-RISK ALLELE'] = gwas['STRONGEST SNP-RISK ALLELE'].astype(str).str.split('; ')
gwas_split = (
    gwas_df
    .withColumn('CHR_ID', f.split('CHR_ID', ';'))
    .withColumn('CHR_POS', f.split('CHR_POS', ';'))
    .withColumn('SNPS', f.split('SNPS', '; '))
    .withColumn('STRONGEST SNP-RISK ALLELE', f.split('STRONGEST SNP-RISK ALLELE', '; '))
)

(
    gwas_split
    .filter(f.size('CHR_POS') != f.size('STRONGEST SNP-RISK ALLELE'))
    .show(10, False, True)
)

-RECORD 0-------------------------------------------------------------------------------------------------------------------------------------------------------
 DATE ADDED TO CATALOG      | 2020-05-19                                                                                                                        
 PUBMEDID                   | 31926482                                                                                                                          
 FIRST AUTHOR               | Shen H                                                                                                                            
 DATE                       | 2020-01-11                                                                                                                        
 JOURNAL                    | Neuropsychopharmacology                                                                                                           
 LINK                       | www.

In [23]:
import pandas as pd

# Load gwas catalog data
gwas_raw = pd.read_csv(
    'gwas_associations_2022-05-20.tsv',
    sep='\t',
    header=0,
    dtype={'P-VALUE': str},
    low_memory=False,
    # na_values=['']
)

# Set a row (association) id -> Seriously... the table is alrady indexed.
gwas_raw['assoc_id'] = list(range(1, gwas_raw.shape[0] + 1))
gwas = gwas_raw.copy()

# Split rows by multiple variants
gwas['CHR_ID'] = gwas['CHR_ID'].astype(str).str.split(';')
gwas['CHR_POS'] = gwas['CHR_POS'].astype(str).str.split(';')
gwas['SNPS'] = gwas['SNPS'].astype(str).str.split('; ')
gwas['STRONGEST SNP-RISK ALLELE'] = gwas['STRONGEST SNP-RISK ALLELE'].astype(str).str.split('; ')

print(len(gwas))

372752


In [54]:
(
    gwas_df
    .filter(f.lower(f.col('SNPS')).contains('chr'))
    .count()
)

10387

In [55]:
10387 / gwas_df.count() * 100

2.7865712323475127

### Ancestries

In [4]:
ancestries = spark.read.csv('gwas_ancestries_2022-05-23.tsv', sep='\t', header=True).persist()
print(ancestries.show(1, False, True))
print(ancestries.count())
print(ancestries.select('STUDY ACCESSION').distinct().count())



-RECORD 0---------------------------------------------------------------------
 STUDY ACCESSION                | GCST009498                                  
 PUBMEDID                       | 31681408                                    
 FIRST AUTHOR                   | Yang XL                                     
 DATE                           | 2019-10-11                                  
 INITIAL SAMPLE DESCRIPTION     | up to 182,902 European ancestry individuals 
 REPLICATION SAMPLE DESCRIPTION | NA                                          
 STAGE                          | initial                                     
 NUMBER OF INDIVDUALS           | 182902                                      
 BROAD ANCESTRAL CATEGORY       | European                                    
 COUNTRY OF ORIGIN              | NR                                          
 COUNTRY OF RECRUITMENT         | NR                                          
 ADDITONAL ANCESTRY DESCRIPTION | null              

In [7]:
import pandas as pd
import requests 


df = 
df.head()

,assoc_files,category,category_index,gc_lambda,num_cases,num_cases_prev,num_controls,num_gw_significant,num_gw_significant_prev,phenocode,phenostring
0,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.98843, '0.01': 0.98437, '0.1': 0.9...",102,87,234437,0,0,AB1_AMOEBIASIS,Amoebiasis
1,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.99329, '0.01': 0.99733, '0.1': 0.9...",1101,891,253346,0,0,AB1_ANOGENITAL_HERPES_SIMPLEX,Anogenital herpesviral [herpes simplex] infection
2,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 1.1145, '0.01': 1.0274, '0.1': 0.982...",1763,1450,258642,4,4,AB1_ARTHROPOD,Arthropod-borne viral fevers and viral haemorr...
3,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.99661, '0.01': 1.0038, '0.1': 0.92...",106,87,255615,0,0,AB1_ASPERGILLOSIS,Aspergillosis
4,[/mnt/nfs/pheweb/r6/generated-by-pheweb/pheno_...,I Certain infectious and parasitic diseases (A...,1,"{'0.001': 0.99495, '0.01': 0.99733, '0.1': 0.9...",176,146,259010,0,0,AB1_ATYPICAL_CNS_VIRUS,Atypical virus infections of central nervous s...


In [16]:
import numpy as np
(
    pd.DataFrame(requests.get('https://r7.finngen.fi/api/phenos').json())
    .loc[lambda df: df.phenostring.isnull()]
)

,assoc_files,category,category_index,gc_lambda,num_cases,num_cases_prev,num_controls,num_gw_significant,num_gw_significant_prev,phenocode,phenostring


In [37]:
from pyspark.sql.window import Window


# from pyspark.sql.functions import row_number
windowSpec  = Window.partitionBy("study").orderBy("pval_text")


(
    spark.createDataFrame([
        {'study':'a', 'pval_text': None},
        {'study':'b', 'pval_text': 'pval 1'},
        {'study':'b', 'pval_text': 'pval 2'},
        {'study':'b', 'pval_text': None},
    ])
    .withColumn("row_number", f.row_number().over(windowSpec) - 1)
    .withColumn(
        "new_study",
        f.when(f.col('row_number') == 0, 'study').otherwise(f.concat_ws('_', 'study', 'row_number'))
    )
    .drop('row_number')
    .show()
)

+---------+-----+---------+
|pval_text|study|new_study|
+---------+-----+---------+
|     null|    b|    study|
|   pval 1|    b|      b_1|
|   pval 2|    b|      b_2|
|     null|    a|    study|
+---------+-----+---------+



In [30]:
def make_new_study_id(df):
    ''' Creates a unique study ID from the origin STUDY ACCESSION for each
        new trait
    Params:
        df (pd.df): dataframe with 2 columns: Study accession, trait name
    Returns:
        list of strs
    '''
    cache = {}
    new_ids = []
    for accession, trait in df.values.tolist():

        # Add missing accession to cache
        if not accession in cache:
            cache[accession] = {'counter': 0}

        # Add missing trait to cache
        if not trait in cache[accession]:
            cache[accession]['counter'] += 1
            cache[accession][trait] = cache[accession]['counter']

        # Create new id. only add suffix if counter > 1
        if cache[accession][trait] == 1:
            new_id = accession
        elif cache[accession][trait] > 1:
            new_id = '{0}_{1}'.format(accession, cache[accession][trait])
        else:
            sys.exit('Error: counter < 1')
        new_ids.append(new_id)

    return new_ids

In [34]:
print(
    pd.DataFrame([
        {'study':'a', 'pval_text': None},
        {'study':'b', 'pval_text': 'pval 1'},
        {'study':'b', 'pval_text': 'pval 2'},
        {'study':'b', 'pval_text': None},
    ])
    .assign(new_study=lambda df: make_new_study_id(df))
    .to_markdown(index=False)
)


| study   | pval_text   | new_study   |
|:--------|:------------|:------------|
| a       |             | a           |
| b       | pval 1      | b           |
| b       | pval 2      | b_2         |
| b       |             | b_3         |


In [53]:
from random import randint, choice
from functools import reduce
import string

def random_char(y):
    """Generate random column names with a given lenght:"""
    return ''.join(choice(string.ascii_letters) for _ in range(y))

def get_dataframe():
    """Generate spark dataframe with some fixed and a random columns and random values"""
    randomcol = random_char(5)
    return spark.createDataFrame([
        {
            'a': randint(-1000, 1000), 
            'b':randint(-1000, 1000),
            randomcol : randint(-1000, 1000)
        } for _ in range(2)
    ])

# Generate a set of random dataframes:
dataframes = [get_dataframe() for _ in range(3)]

# Merging dataframes using reduce:
merged = reduce(lambda df1, df2: df1.unionByName(df2, allowMissingColumns=True), dataframes)
merged.show()

# +-----+----+----+-----+-----+
# |Nzbke|   a|   b|iFRbE|UtPKj|
# +-----+----+----+-----+-----+
# | -702| 994| 544| null| null|
# |    0| 700| 922| null| null|
# | null|-697| 434|  314| null|
# | null|-752| 165| -986| null|
# | null|-747|-227| null|  133|
# | null|-230| 517| null| -768|
# +-----+----+----+-----+-----+

In [54]:
merged.show()

+-----+----+----+-----+-----+
|Nzbke|   a|   b|iFRbE|UtPKj|
+-----+----+----+-----+-----+
| -702| 994| 544| null| null|
|    0| 700| 922| null| null|
| null|-697| 434|  314| null|
| null|-752| 165| -986| null|
| null|-747|-227| null|  133|
| null|-230| 517| null| -768|
+-----+----+----+-----+-----+

